In [1]:
# import pymysql.cursors
# import pandas as pd
# connection = pymysql.connect(host='ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com', 
#                              user='read_only', password='WHC63fKyjudfSz2U', db='CAPC_APIGATEWAY',
#                              cursorclass=pymysql.cursors.SSDictCursor)

In [2]:
import pandas as pd
from sqlalchemy import create_engine

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# # stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [3]:
import os

In [4]:
# connection

In [5]:
with open('/home/trishalaswain/data_for_analysis/facilities.txt') as file:
    lines = [line.rstrip()+'-' for line in file]

In [6]:
lines

['SCP-488965-',
 'SCP-1097-',
 'SCP-1113-',
 'SCP-8-',
 'SCP-4594-',
 'SCP-194-',
 'SCP-5365-',
 'SCP-5381-',
 'SCP-5483-',
 'SCP-6068-',
 'SCP-6074-',
 'SCP-6126-',
 'SCP-6182-',
 'SCP-8804-',
 'SCP-8816-',
 'SCP-72395-',
 'SCP-471679-',
 'SCP-490820-']

In [7]:
path = '/home/trishalaswain/Downloads/Feb_01_to_28_tsv/'
list_files = os.listdir(path)
for item in lines:
    for file in list_files:
        if item in file and str(file).endswith('28.tsv'):
            df = pd.read_table(path+file, index_col=False)
            print(path+file)
            acc = df.accountNumber.tolist()
            list_string = map(str, acc)
            acc = list(list_string)
            # sql = """SELECT DISTINCT
            # am.client_id,
            # am.facility_id,
            # dpd.eandmcs_request_id,
            # am.account_number,
            # dpd.nlp_pcs_xml_result_s3_path,
            #    dpd.nlp_cdp_xml_result_s3_path,
            #    dm.service_date
            # FROM
            #     account_mst am
            #         INNER JOIN
            #     document_mst dm ON am.id = dm.account_id and dm.is_physician_document = 1
            #         INNER JOIN
            #     document_processing_detail dpd ON dm.id = dpd.document_id
            # WHERE
            #     am.account_number in ({}) 
            # order by dpd.created_date desc;""".format(",".join(str(x) for x in acc))
            
            df_all=[]


            query='''
            SELECT DISTINCT
                am.account_number,
                dpd.nlp_request_s3_path,
                dpd.nlp_ner_entities_result_s3_path,
                am.facility_id,
                dm.service_date,
                dpd.nlp_pcs_xml_result_s3_path,
                dpd.preprocessing_request_s3_path,
                dpd.preprocessing_result_s3_path,
                dm.orignal_path,
                dm.type_id,
                dpd.created_date,
                dpd.ocr_request_s3_path,
                dpd.ocr_result_s3_path,
                dpd.eandmcs_request_s3_path,
                dpd.eandmcs_result_s3_path,
                dpd.ruleengine_request_s3_path,
                dpd.cptcs_request_s3_path,
                dpd.cptcs_result_s3_path,
                dm.account_id,
                em.id as encounter_id,
                dc.code AS suggested_code,
                dpd.id AS DPD_ID,
                dm.path as filePath,
                ec.code AS final_code
            FROM
                account_mst am
                    JOIN
                document_mst dm ON dm.account_id = am.id
                    JOIN
                document_processing_detail dpd ON dpd.document_id = dm.id
                    INNER JOIN
                encounter_mst em ON am.id = em.account_id
                    left JOIN
                document_code dc ON dc.document_id = dm.id
                    left JOIN
                eandm_code ec ON ec.encounter_id = em.id
            WHERE
                account_number IN ({})
                    AND dc.code_category = 'EANDM'
                    and is_physician_document=1
                    and dm.is_active=1
                    and am.is_active=1
            ORDER BY dpd.created_date DESC;
            '''.format(','.join(["'"+str(r)+"'" for r in acc]))

            # print(query)
            df_all.append(pd.read_sql(query, con=db_connection))
            # print(f_id)
            df_query=pd.concat(df_all)
            df_query.account_number=df_query.account_number.apply(lambda x:str(x))
            df_query=df_query.groupby(["account_number", "service_date", 'facility_id']).first().reset_index()
            df_query=df_query.sort_values(['facility_id', 'type_id'])
            cmd=[]
            fac_list=[]

            for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id,sugg, final,encounter_id, t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                                  df_query.nlp_pcs_xml_result_s3_path,
                                                                     df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                                     df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                                     df_query.account_id, df_query.type_id, 
                                                                     df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                                     df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                                     df_query.ruleengine_request_s3_path,
                                                                     df_query.encounter_id,
                                                                     df_query.suggested_code, df_query.final_code,
                                                                     df_query.encounter_id,
                                                                     df_query.type_id,
                                                                     df_query.filePath
                                                                    ):
                # if pcs == None or acc==None:
                #     continue
                fac_name=file_path.split("/")[2]
                cli=file_path.split("/")[0]
                fac_list.append(fac_name)

                # eandm
                cmd.append("aws s3 cp s3://ezdi-{}-bucket/{} {}/{}/{}_{}_{}_pcs.xml".format(env, pcs, fac_name,enc_id, acc, sugg, final))
                # cmd.append("aws s3 cp s3://ezdi-{}-bucket/{} {}_{}_{}_pcs.xml".format(env, pcs, encounter_id, sugg, final))

                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_eandm_request.json".format(env, em_req,fac_name, enc_id, acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_eandm_result.json".format(env, em_res,fac_name, acc_id, acc))

                # cpt
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.xml".format(env, pcs, fac_name, enc_id, acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_cpt_result.json".format(env, ctp_res,fac_name, acc_id, acc))

                # preprocess
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

                # original path
                # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.pdf".format(env,ori_path,fac_name,acc_id,acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

                # ocr
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

                # nlp
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc))
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))

                # rule
                # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

                # print(pre)
                # print(pcs, acc)


                # print(re.split('/\d+_\d+_\d+', pcs))
            
            
#             sql_query = pd.read_sql(sql,con= connection)
            
#             sql_query.isna().sum()
            
#             sql_query=sql_query.dropna()
            
#             cmd1 = []

#             for client_id,facility_id,eandmcs_request_id,account_number in zip(sql_query.client_id,sql_query.facility_id,sql_query.eandmcs_request_id,sql_query.account_number):
#                 # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(eandmcs)+"    "+ str(enc_id) + "_request.json")
#                 cmd1.append("aws s3 cp s3://ezdi-production-bucket/paas/eandmcs-service/data/"+str(client_id)+"/"+str(facility_id)+"/"+str(eandmcs_request_id)+ "    "+str(account_number)+"/"+"."+" "+"--recursive --exclude \"*cdp*\"")
                
            # open file in write mode
            with open(r'/home/trishalaswain/Downloads/scripts2/' + item.rstrip('-') + '.sh', 'w') as fp:
                for item in cmd:
                    # write each item on a new line
                    fp.write("%s\n" % item)
                print('Done')

/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-488965-ADVENTHEALTH FOUR CORNERS ER-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-1097-ADVENTHEALTH HEART OF FLORIDA-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-1113-ADVENTHEALTH LAKE WALES-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-8-OCHSNER ABROM KAPLAN MEM HOSPITAL-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-4594-MONTPELIER HOSPITAL-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-194-CHI ST. VINCENT HOT SPRINGS-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-5365-GREENVILLE COMMUNITY HOSPITAL EAST-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-5381-INDIAN PATH COMMUNITY HOSPITAL-2023-02-01-2023-02-28.tsv
Done
/home/trishalaswain/Downloads/Feb_01_to_28_tsv/SCP-5483-JOHNSON COUNTY COMM